In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
df=yf.download("RELIANCE.NS",period="2y")
df.head()

/tmp/ipython-input-1768133859.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download("RELIANCE.NS",period="2y")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2023-12-19,1269.652710,1277.494571,1253.323497,1268.114050,16820604
2023-12-20,1254.291382,1289.878086,1250.742696,1276.080186,17736722
2023-12-21,1271.861450,1277.296201,1249.874163,1254.217020,13765454
2023-12-22,1273.102295,1280.968997,1264.466141,1270.397340,16541784
2023-12-26,1279.554443,1286.453333,1271.935784,1274.566338,7465664


In [3]:
def sma(data,window=20):
  return data["Close"].rolling(window=window).mean()

In [4]:
def ema(data,window=20):
  return data["Close"].ewm(span=window,adjust=False).mean()

In [5]:
def macd(data):
  ema_12=data["Close"].ewm(span=12,adjust=False).mean()
  ema_26=data["Close"].ewm(span=26,adjust=False).mean()
  macd_line=ema_12-ema_26
  signal_line = macd_line.ewm(span=9, adjust=False).mean()
  return macd_line,signal_line

In [6]:
def rsi(data,window=14):
  delta=data["Close"].diff()
  gain=(delta.where(delta>0,0)).rolling(window=window).mean()
  loss=(-delta.where(delta<0,0)).rolling(window=window).mean()
  rs=gain/loss
  return 100-(100/(1+rs))

In [7]:
def bollinger(data,window=20):
  sma=data["Close"].rolling(window=window).mean()
  std=data["Close"].rolling(window=window).std()
  upper_band=sma+(2*std)
  lower_band=sma-(2*std)
  return upper_band,lower_band

In [8]:
def stochastic(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    k_percent = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    d_percent = k_percent.rolling(window=3).mean()
    return k_percent, d_percent

In [9]:
df['SMA_50']=sma(df,50)
df['EMA_20']=ema(df,20)
df['MACD'],df['Signal_line']=macd(df)
df['RSI']=rsi(df)
df['Upper_band'],df['Lower_band']=bollinger(df)
df['Stoch_K'], df['Stoch_D'] = stochastic(df)


In [10]:
print(df[['Close', 'SMA_50', 'RSI', 'MACD', 'Stoch_K']].tail())

Price             Close       SMA_50        RSI       MACD    Stoch_K
Ticker      RELIANCE.NS                                              
Date                                                                 
2025-12-15  1556.199951  1489.224006  56.812546  15.168064  60.596438
2025-12-16  1542.300049  1492.570007  36.832064  13.833758  38.775581
2025-12-17  1544.400024  1495.762007  40.537839  12.798231  42.072242
2025-12-18  1544.400024  1499.302007  38.006234  11.841073  44.741299
2025-12-19  1565.099976  1503.048005  49.567474  12.607499  83.922298
